# Librerías

In [ ]:
'''Importar las librerías necesarias'''
import h5py
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import cross_validate, KFold
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict

In [ ]:
'''Ruta con los archivos de la data ya procesada en formato H5'''
test_size    = 0.20
seed         = 9
h5_color     = './color_features.h5'
h5_colorLb   = './color_features_labels.h5'
h5_texture   = './textural_features.h5'
h5_textureLb = './textural_features_labels.h5'
scoring      = {'accuracy' : make_scorer(accuracy_score),
                'f1_score' : make_scorer(f1_score, average = 'weighted'), 
                'precision' : make_scorer(precision_score, average = 'weighted'),
                'recall' : make_scorer(recall_score, average = 'weighted')}

# Load Data

In [ ]:
'''Importar el vector de características y las etiquetas entrenadas'''
h5f_color   = h5py.File(h5_color, 'r')
h5f_colorLb = h5py.File(h5_colorLb, 'r')

color_features_string          = h5f_color['Color_Features_Granadilla']
color_features_labels_string   = h5f_colorLb['Color_Features_Labels_Granadilla']

color_features = np.array(color_features_string)
color_features_labels  = np.array(color_features_labels_string)

h5f_color.close()
h5f_colorLb.close()

'''Verificar la forma del vector de características y las etiquetas'''
print("[STATUS] features shape: {}".format(color_features.shape))
print("[STATUS] labels shape: {}".format(color_features_labels.shape))

print("[STATUS] training started...")

[STATUS] features shape: (5056, 512)
[STATUS] labels shape: (5056,)
[STATUS] training started...


In [ ]:
'''Importar el vector de características y las etiquetas entrenadas'''
h5f_texture  = h5py.File(h5_texture, 'r')
h5f_textureLb = h5py.File(h5_textureLb, 'r')

textural_features_string          = h5f_texture['Textural_Features_Granadilla']
textural_features_labels_string   = h5f_textureLb['Textural_Features_Labels_Granadilla']

textural_features = np.array(textural_features_string )
textural_features_labels  = np.array(textural_features_labels_string)

h5f_texture.close()
h5f_textureLb.close()

'''Verificar la forma del vector de características y las etiquetas'''
print("[STATUS] features shape: {}".format(textural_features.shape))
print("[STATUS] labels shape: {}".format(textural_features_labels.shape))

print("[STATUS] training started...")

[STATUS] features shape: (5056, 13)
[STATUS] labels shape: (5056,)
[STATUS] training started...


In [ ]:
'''Definir en una variable las características globales'''
global_features = np.hstack([color_features, textural_features])
print("[STATUS] features shape: {}".format(global_features.shape))
print("[STATUS] labels shape: {}".format(textural_features_labels.shape))

[STATUS] features shape: (5056, 525)
[STATUS] labels shape: (5056,)


# Identify X and Y

In [ ]:
'''Identificar las etiquetas de entrada y salida X y Y'''
X = global_features
Y = color_features_labels

# Training and Testing

In [ ]:
'''Dividir el conjunto de datos en entrenamiento y prueba'''
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                        test_size   = test_size,
                                        random_state = seed)

X_train.shape, X_test.shape

((4044, 525), (1012, 525))

# SVM

In [ ]:
'''Establecer los parámetros para aplicar kernel lineal'''
linear_param_grid = {
    
        "kernel": ["linear"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [ ]:
'''Establecer los parámetros para aplicar kernel polinomial'''
poly_param_grid = {
    
        "kernel": ["poly"],
        "degree": [2, 3, 4],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

In [ ]:
'''Establecer los parámetros para aplicar kernel RBF'''
rbf_param_grid = {
        "kernel": ["rbf"],
        "gamma": ["auto", "scale"],
        "C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000],
        "decision_function_shape": ['ovo', 'ovr']
}

# Linear Kernel

In [ ]:
'''Crear un objeto GridSearchCV y ajustarlo a los datos de entrenamiento'''
linear_grid = GridSearchCV(svm.SVC(), linear_param_grid)
linear_grid.fit(X_train,y_train)
print(linear_grid.best_estimator_)

SVC(C=10, decision_function_shape='ovo', kernel='linear')


In [ ]:
'''Predecir el objetivo para el conjunto de datos de prueba y evaluar el modelo, mediante las diferentes métricas'''
linear_pred = linear_grid.predict(X_test)
linear_accuracy = accuracy_score(y_test, linear_pred)
linear_precision = precision_score(y_test, linear_pred, average='weighted')
linear_recall = recall_score(y_test, linear_pred, average='weighted')
linear_f1 = f1_score(y_test, linear_pred, average='weighted')
print('Accuracy (Linear Kernel): ', "%.2f" % (linear_accuracy*100))
print('F1 (Linear Kernel): ', "%.2f" % (linear_f1*100))
print('Precision (Linear Kernel): ', "%.2f" % (linear_precision*100))
print('Recall (Linear Kernel): ', "%.2f" % (linear_recall*100))

Accuracy (Linear Kernel):  85.08
F1 (Linear Kernel):  84.79
Precision (Linear Kernel):  85.06
Recall (Linear Kernel):  85.08


# Polynomial Kernel

In [ ]:
'''Crear un objeto GridSearchCV y ajustarlo a los datos de entrenamiento'''
poly_grid = GridSearchCV(svm.SVC(), poly_param_grid)
poly_grid.fit(X_train,y_train)
print(poly_grid.best_estimator_)

SVC(C=100, decision_function_shape='ovo', degree=2, kernel='poly')


In [ ]:
'''Predecir el objetivo para el conjunto de datos de prueba y evaluar el modelo, mediante las diferentes métricas'''
poly_pred = poly_grid.predict(X_test)
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_precision = precision_score(y_test, poly_pred, average='weighted')
poly_recall = recall_score(y_test, poly_pred, average='weighted')
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))
print('Precision (Polynomial Kernel): ', "%.2f" % (poly_precision*100))
print('Recall (Polynomial Kernel): ', "%.2f" % (poly_recall*100))

Accuracy (Polynomial Kernel):  89.72
F1 (Polynomial Kernel):  89.53
Precision (Polynomial Kernel):  89.91
Recall (Polynomial Kernel):  89.72


# RBF Kernel

In [ ]:
'''Crear un objeto GridSearchCV y ajustarlo a los datos de entrenamiento'''
rbf_grid = GridSearchCV(svm.SVC(probability=True), rbf_param_grid)
rbf_grid.fit(X_train,y_train)
print(rbf_grid.best_estimator_)

SVC(C=100, decision_function_shape='ovo', probability=True)


In [ ]:
'''Predecir el objetivo para el conjunto de datos de prueba y evaluar el modelo, mediante las diferentes métricas'''
rbf_pred = rbf_grid.predict(X_test)
rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_precision = precision_score(y_test, rbf_pred, average='weighted')
rbf_recall = recall_score(y_test, rbf_pred, average='weighted')
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))
print('Precision (RBF Kernel): ', "%.2f" % (rbf_precision*100))
print('Recall (RBF Kernel): ', "%.2f" % (rbf_recall*100))

Accuracy (RBF Kernel):  89.62
F1 (RBF Kernel):  89.48
Precision (RBF Kernel):  89.67
Recall (RBF Kernel):  89.62
